In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_json('dataset/datasets/test_qa.jsonl', lines=True)

In [3]:
df.head(5)

,id,name,conversations
0,"[57335, 57342, 57349, 57356, 57365, 57371, 573...","[57335.csv, 57342.csv, 57349.csv, 57356.csv, 5...","[{'from': 'human', 'stage': '3', 'attribute': ..."
1,75655,75655.csv,"[{'from': 'human', 'stage': '1', 'attribute': ..."
2,80665,80665.csv,"[{'from': 'human', 'stage': '1', 'attribute': ..."
3,"[43212, 43217, 43226, 43232, 43238, 43246, 432...","[43212.csv, 43217.csv, 43226.csv, 43232.csv, 4...","[{'from': 'human', 'stage': '3', 'attribute': ..."
4,"[28766, 28782, 28797, 28839, 28868, 28890, 289...","[28766.csv, 28782.csv, 28797.csv, 28839.csv, 2...","[{'from': 'human', 'stage': '3', 'attribute': ..."


In [4]:
for element in df.loc[0]['conversations']:
    print(element)

{'from': 'human', 'stage': '3', 'attribute': 'close', 'value': 'Given the time series signal <ts>, by perceiving the engine signal across 10 cycles to reflect different health states and performing temporal inference to predict the health decline trend, what is the qualitative condition of the engine? a:  Good Condition\n b: Moderate Condition\n c: Poor Condition\n d: Bad Condition\n e: Extremely Poor Condition\n'}
{'from': 'gpt', 'value': 'd'}
{'from': 'human', 'stage': '3', 'attribute': 'close', 'value': 'Given the time series signal <ts>, by perceiving the engine signal across 10 cycles to reflect different health states and performing temporal inference to predict the health decline trend, what is the probability range of engine failure? a: 1%-10%\n b: 10%-30%\n c: 30%-50%\n d: 50%-70%\n e: 70%-100%\n'}
{'from': 'gpt', 'value': 'd'}
{'from': 'human', 'stage': '4', 'attribute': 'open', 'value': "In light of the engine signal data collected over a span of 10 operational cycles, what 

In [5]:
# df[:2].to_json('dataset/datasets/test_qa_2.jsonl', orient='records', lines=True)

In [2]:
# df_infer_metrics = pd.read_json('dataset/datasets/infer_metrics.json')
df_inference_results = pd.read_json('dataset/datasets/inference_results_infer_20251219_214118.json')
df_model_output = pd.read_json('dataset/datasets/model_output.json')
df_samples = pd.read_json('dataset/datasets/samples_json.jsonl', lines=True)
df_samples_ts = pd.read_json('dataset/datasets/samples_ts_json.jsonl', lines=True)

In [14]:
df_inference_results.head()

,index,stage,input,prediction,label,is_correct
0,0,3,system\nYou are a helpful assistant.\nuser\nGi...,d,d,True
1,0,3,system\nYou are a helpful assistant.\nuser\nGi...,d,d,True
2,0,4,system\nYou are a helpful assistant.\nuser\nIn...,It is imperative to conduct a comprehensive as...,It is critical to promptly repair or replace a...,False
3,1,1,system\nYou are a helpful assistant.\nuser\nMa...,The altitude exhibits a gradual decrease. This...,The altitude is observed to be rising steadily...,False
4,1,1,system\nYou are a helpful assistant.\nuser\nIn...,The Throttle Resolver Angle exhibits a gradual...,The Throttle Resolver Angle exhibits a steady ...,False


In [16]:
df_samples.head()

,id,stage,form,question,answer,line_num
0,"[28766, 28782, 28797, 28839, 28868, 28890, 289...",3,close,"Given the time series signal <ts>, by perceivi...",c,4
1,"[28766, 28782, 28797, 28839, 28868, 28890, 289...",3,close,"Given the time series signal <ts>, by perceivi...",c,4
2,75655,2,close,What is the health status of LPC of the given ...,a,1
3,80665,1,open,In the context of the provided engine signal o...,The Total Temperature at the HPT outlet remain...,2
4,"[57335, 57342, 57349, 57356, 57365, 57371, 573...",3,close,"Given the time series signal <ts>, by perceivi...",d,0


In [17]:
df_samples_ts.head()

,form,stage,query_ids,input_ids,labels,ts_values,index
0,close,3,"[22043, 279, 882, 4013, 8286, 366, 2576, 8066,...","[151644, 8948, 198, 2610, 525, 264, 10950, 178...","[66, 151645]","[[-0.698168754577636, -0.32407632470130904, -1...",4
1,close,3,"[22043, 279, 882, 4013, 8286, 366, 2576, 8066,...","[151644, 8948, 198, 2610, 525, 264, 10950, 178...","[66, 151645]","[[-0.698168754577636, -0.32407632470130904, -1...",4
2,close,2,"[3838, 374, 279, 2820, 2639, 315, 49075, 315, ...","[151644, 8948, 198, 2610, 525, 264, 10950, 178...","[64, 151645]","[[-0.7759246230125421, -0.672634720802307, 0.9...",1
3,open,1,"[641, 279, 2266, 315, 279, 3897, 4712, 8286, 9...","[151644, 8948, 198, 2610, 525, 264, 10950, 178...","[785, 10657, 37022, 518, 279, 472, 2828, 26389...","[[1.433789014816284, 1.073679566383361, 0.9743...",2
4,close,3,"[22043, 279, 882, 4013, 8286, 366, 2576, 8066,...","[151644, 8948, 198, 2610, 525, 264, 10950, 178...","[67, 151645]","[[-0.814841032028198, -0.712989151477813, -1.8...",0


In [29]:
np.array(df_samples_ts.loc[0]['ts_values']).shape

(600, 33)

In [3]:
list_of_samples = df_samples_ts.to_dict(orient="records")

In [4]:
class Args():
    def __init__(self):
        self.config = 'yaml/infer.yaml'
        self.output_dir = 'inference_results'
        self.model_checkpoint = 'checkpoints/ITFormer-0.5B'
        self.seed = 42
        self.batch_size = 12
        self.num_workers = 4
        self.max_new_tokens = 128
        self.sample_data = list_of_samples

args = Args()


In [5]:
from inference import main_inference
main_inference(args)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

W1220 04:34:13.435190 10028 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🚀 Starting inference process...
🔧 Using config file: yaml/infer.yaml
💾 Results will be saved to: inference_results

⚙️ Loading model from checkpoint...
🔗 Using LLM model: LLM/Qwen2.5-0.5B-Instruct
✅ Loaded LLM model from: LLM/Qwen2.5-0.5B-Instruct
🧊 Model set to inference mode - all parameters frozen
Loading model weights from: checkpoints/ITFormer-0.5B\model.safetensors
Found 290 LLM weights (will be ignored)
Found 150 non-LLM weights (will be loaded)
LLM missing keys (ignored): 291 keys

🔢 Model parameter counts (in M):
   LLM:         494.03M
   ITFormer:    47.91M
   TS_Encoder:  16.87M
   Total:       560.88M
✅ Model loaded successfully!

📊 Preparing test dataset...
Loading tokenizer from checkpoint
📁 Test set size: 102 samples
🔢 Batch size: 12, Total batches: 9

🔍 Starting test set inference...


Inference progress: 100%|██████████| 9/9 [02:08<00:00, 14.24s/it]



📊 Calculating evaluation metrics...

📈 Evaluation results:

🔹 Stage 1 metrics:
   BLEU: 0.2091
   rouge1: 0.4827
   rouge2: 0.2785
   rougeL: 0.4303

🔹 Stage 2 metrics:
   precision: 0.6389
   recall: 0.4694
   f1: 0.5412
   exact_match_accuracy: 0.5476

🔹 Stage 3 metrics:
   precision: 0.3750
   recall: 0.3750
   f1: 0.3750
   exact_match_accuracy: 0.3750

🔹 Stage 4 metrics:
   BLEU: 0.0946
   rouge1: 0.4828
   rouge2: 0.1768
   rougeL: 0.3937

✅ Inference results saved to: inference_results\inference_results_infer_20251220_043628.json

💾 Evaluation metrics saved to: inference_results\infer_metrics.json


In [33]:
torch.cuda.is_available()

True